<a href="https://colab.research.google.com/github/Habtamuyihun561/MAIA/blob/main/AIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
from google.colab.patches import cv2_imshow
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage import exposure, io
from skimage import exposure
from skimage.restoration import denoise_nl_means, estimate_sigma
from typing import List

In Python, the notation List[np.ndarray] -> np.ndarray is a type hint often used in function signatures to specify the types of arguments and the return type of a function. Let's break down the meaning:

List[np.ndarray]: This specifies that the function expects an argument which is a list, where each element of the list is an instance of np.ndarray. np.ndarray is a type provided by the NumPy library, commonly used for handling arrays (n-dimensional arrays). Therefore, List[np.ndarray] indicates a list of these arrays.

-> np.ndarray: This part of the notation indicates the return type of the function. It tells you that the function is expected to return an np.ndarray object.

Combining these, the entire type hint List[np.ndarray] -> np.ndarray is commonly found in the definition of a function that takes a list of np.ndarray objects as input and returns a single np.ndarray object. Here’s a simple example to illustrate this:

In [ ]:
# @title Default title text
# tip codes:
import numpy as np

def combine_arrays(arrays: List[np.ndarray]) -> np.ndarray:
    """Combine a list of numpy arrays into a single array."""
    return np.concatenate(arrays)

# Example usage
array1 = np.array([1, 2, 3])
array2 = np.array([4, 5, 6])
result = combine_arrays([array1, array2])
print(result)


[1 2 3 4 5 6]


In [ ]:
pip install --upgrade numpy opencv-python


In [3]:

def load_images_for_reference(image_dir, image_format=".JPG"):
    """Load images to determine the reference image based on median brightness."""
    brightness = []
    for filename in os.listdir(image_dir):
        if filename.endswith(image_format):
            img_path = os.path.join(image_dir, filename)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if image is not None:
                # Calculate and store brightness along with the image path
                current_brightness = np.mean(image)
                brightness.append((current_brightness, img_path))

    # Determine the reference image by median brightness
    median_brightness = np.median([b[0] for b in brightness])
    reference_image_path = min(brightness, key=lambda x: abs(x[0] - median_brightness))[1]
    return cv2.imread(reference_image_path, cv2.IMREAD_GRAYSCALE)

**Tip**
closest_img = min(images, key=lambda x: float(abs(np.mean(x) - median_brightness)))

images: This is an iterable (probably a list) containing items that are likely numpy arrays representing images. Each x in images is one of these arrays.

min() Function: The min() function is used to find the element in the iterable images that has the smallest value of the key specified. The key is a function that determines how the "smallness" is calculated for each element.

Lambda Function lambda x: float(abs(np.mean(x) - median_brightness)):

lambda x: This is a lambda function that takes an argument x, where x is an individual element from images.
np.mean(x): This computes the mean (average) brightness of the image x. Assuming x is a NumPy array, np.mean(x) will calculate the average value of all pixels if x is a grayscale image, or the average across all pixels and color channels if x is a color image.
median_brightness: This is a variable (not defined in the snippet) which likely holds the median brightness of some collection of images or a target brightness value that you want to match.
abs(np.mean(x) - median_brightness): This calculates the absolute difference between the mean brightness of the image x and median_brightness. It measures how far the average brightness of the image x is from the median_brightness.
float(...): This ensures that the result is a floating-point number. This is somewhat redundant here because the result of abs with NumPy operations would already be a float, but it ensures type consistency.
Finding the Closest Image: The min() function uses the lambda function as a key to find the image x in images whose mean brightness is closest to the median brightness (i.e., the image for which the absolute difference between its mean brightness and the median brightness is the smallest).

closest_img: This variable will store the image from images that has the mean brightness closest to median_brightness.

The code is effectively selecting the image from a set that is most similar in brightness to a specified median brightness, which is useful in applications where consistency in brightness among images is desired.

In [4]:
def batch_process_images(image_dir, reference_image, histogram_match_dir,denoised_dir, batch_size=10):
    """Process images in batches using the determined reference image and save them."""
    filenames = [f for f in os.listdir(image_dir) if f.endswith('.JPG')]
    for i in range(0, len(filenames), batch_size):
        batch_files = filenames[i:i+batch_size]
        batch_images = []
        for f in batch_files:
            img_path = os.path.join(image_dir, f)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if image is not None:
                image = image.astype(np.uint8)  # Convert image to uint8
                batch_images.append(image)
        matched_images = [histogram_matching(img, reference_image) for img in batch_images]
        processed_images = [apply_non_local_means(unsharp_mask(apply_clahe(img))) for img in matched_images]

        save_images(matched_images, batch_files, histogram_match_dir)
        save_images(processed_images, batch_files, denoised_dir)

In [5]:
def histogram_matching(source_image, reference_image):
    """Apply histogram matching from skimage."""
    return exposure.match_histograms(source_image, reference_image, channel_axis=None)

In [6]:
def apply_clahe(image):
    """Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) to an image."""
    if image.dtype != np.uint8:
        image = np.uint8(image)  # Ensure image is in uint8 format
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(image)


In [7]:
def unsharp_mask(image, sigma=0.25, strength=0.5):
    """Applies unsharp mask to enhance image sharpness."""
    blurred = cv2.GaussianBlur(image, (0, 0), sigma)
    sharpened = cv2.addWeighted(image, 1.0 + strength, blurred, -strength, 0)
    return sharpened

In [8]:
def apply_non_local_means(image):
    """Apply non-local means denoising."""
    return cv2.fastNlMeansDenoising(image, None, 12, 9, 23)

In [9]:
def save_images(images, filenames, output_dir):
    """Save processed images to the specified directory using their original filenames."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for img, filename in zip(images, filenames):
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, img.astype(np.uint8))

In [10]:
radiograph_img_dir= "/content/drive/My Drive/AIA/Radiographs"

In [ ]:

reference_image = load_images_for_reference(radiograph_img_dir)

matched_img="/content/drive/My Drive/AIA/Histogram_Matched"

denoised_img="/content/drive/My Drive/AIA/Denoised_CLAHE"


In [ ]:
batch_process_images(radiograph_img_dir, reference_image,matched_img ,denoised_img, batch_size=10)

This line of code uses the match_histograms function, which is likely from the skimage.exposure module (part of the scikit-image library in Python). The function is used to transform the histogram of an input image (source_image) so that its histogram matches that of a reference image (reference_image).

The parameter channel_axis specifies the axis of the array that corresponds to the color channels in the images:

channel_axis=None: This implies that the images are treated as grayscale images, where there is no specific axis for color channels. Therefore, the entire image is processed as a single channel.

If channel_axis is specified (e.g., channel_axis=-1 or channel_axis=2): This tells the function that the images are in a format where one of the axes corresponds to different color channels (commonly, RGB images). Specifying this parameter allows the function to apply histogram matching separately for each channel of the images. For example, in a typical color image with shape (height, width, channels), the channel_axis would be -1 or 2, indicating that the channels (like RGB) are on the last axis.

In summary, the channel_axis parameter helps the match_histograms function understand whether it is dealing with grayscale images or color images with specific channels, and how to appropriately apply histogram matching to them.

#Segmentation

In [64]:
def load_images_from_folder(folder, batch_size=50):
    images = []
    filenames = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
            filenames.append(filename)
            if len(images) == batch_size:
                yield images, filenames  # Use yield to generate a batch
                images = []  # Reset for next batch
                filenames = []
    if images:  # Check if there are leftover images after the last full batch
        yield images, filenames



In [ ]:
# @title Default title text

# Example Usage
folder_path = 'path/to/your/images'
for batch_images, batch_filenames in load_images_from_folder(folder_path, batch_size=50):
    print(f"Processed a batch of {len(batch_images)} images.")

In [65]:
def save_images(images, filenames, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    for img, filename in zip(images, filenames):
        cv2.imwrite(os.path.join(folder, filename), img)

In [66]:


def adaptive_mean_thresholding(images):
    thresholded_images = []
    for image in images:
        thresh = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                       cv2.THRESH_BINARY, 115, -8)#87, -3, 97
        thresholded_images.append(thresh)
    return thresholded_images

In [67]:
def adaptive_gaussian_thresholding(images):
    thresholded_images = []
    for image in images:
        thresh = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY, 125,-8)#67, 3
        thresholded_images.append(thresh)
    return thresholded_images

In [68]:
def adaptive_niblack_thresholding(images):
    from skimage.filters import threshold_niblack
    thresholded_images = []
    for image in images:
        thresh = threshold_niblack(image,window_size=115, k=-0.4 )#window_size=155, k=0.1
        binary_image = image > thresh
        thresholded_images.append((binary_image * 255).astype(np.uint8))
    return thresholded_images


In [69]:

def otsu_thresholding(images):
    thresholded_images = []
    for image in images:
        _, thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        thresholded_images.append(thresh)
    return thresholded_images

In [70]:

def process_images(source_folder, save_folder_base, thresholding_function):
    for images, filenames in load_images_from_folder(source_folder):
        # Apply thresholding function
        thresholded_images = thresholding_function(images)
        # Save the results
        save_folder = os.path.join(save_folder_base, thresholding_function.__name__)
        save_images(thresholded_images, filenames, save_folder)

In [71]:

source_folder = "/content/drive/My Drive/AIA/Denoised_CLAHE"
save_folder_base = "/content/drive/My Drive/AIA"


In [72]:
process_images(source_folder, save_folder_base, adaptive_mean_thresholding)

In [73]:
process_images(source_folder, save_folder_base, adaptive_gaussian_thresholding)

In [74]:
process_images(source_folder, save_folder_base, adaptive_niblack_thresholding)

In [ ]:
process_images(source_folder, save_folder_base, otsu_thresholding)


#Apply Masking

In [75]:
def load_images_from_folder(folder, batch_size=50):
    """ Generator to yield batches of images and their filenames. """
    images = []
    filenames = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if image is not None:
            images.append(image)
            filenames.append(filename)
            if len(images) == batch_size:
                yield images, filenames
                images, filenames = [], []
    if images:
        yield images, filenames

In [76]:
def save_images(images, filenames, folder):
    """ Save processed images to a specified folder. """
    if not os.path.exists(folder):
        os.makedirs(folder)
    for img, filename in zip(images, filenames):
        cv2.imwrite(os.path.join(folder, filename), img)

In [77]:
def create_elliptical_mask(image):
    """ Create an elliptical mask for given image dimensions. """
    height, width = image.shape
    center = (width // 2, round(height * 0.6))  # Center of the ellipse
    axes = (width // 3, height // 3)  # Semi-major and semi-minor axes

    # Create a blank mask and draw the ellipse
    mask = np.zeros_like(image, dtype=np.uint8)
    cv2.ellipse(mask, center, axes, 0, 0, 360, 1, -1)  # Angle 0, startAngle 0, endAngle 360, filled

    # Optionally, remove the upper unwanted part
    cv2.rectangle(mask, (0, 0), (width, center[1] - axes[1]), 0, -1)  # Adjust rectangle to remove top part
    return mask

In [78]:
def apply_mask_to_images(image_folder, save_folder, batch_size=50):
    """ Apply a mask to images in batches and save the results. """
    for images, filenames in load_images_from_folder(image_folder, batch_size):
        masked_images = []
        for image in images:
            mask = create_elliptical_mask(image)  # Create a mask for each image
            masked_image = cv2.bitwise_and(image, image, mask=mask)  # Apply the mask to the image
            masked_images.append(masked_image)
        save_images(masked_images, filenames, save_folder)

In [54]:
# Segmentation folders and corresponding save folders
segmentation_folders = {
    "/content/drive/My Drive/AIA/adaptive_mean_thresholding": "/content/drive/My Drive/AIA/mask/adaptive_mean_thresholding",
    "/content/drive/My Drive/AIA/adaptive_gaussian_thresholding": "/content/drive/My Drive/AIA/mask/adaptive_gaussian_thresholding",
    "/content/drive/My Drive/AIA/adaptive_niblack_thresholding": "/content/drive/My Drive/AIA/mask/adaptive_niblack_thresholding",
    "/content/drive/My Drive/AIA/otsu_thresholding": "/content/drive/My Drive/AIA/mask/otsu_thresholding"
}

In [79]:



# Apply mask to each segmentation result folder
for input_folder, output_folder in segmentation_folders.items():
    apply_mask_to_images(input_folder, output_folder)


#Performance Metrics

In [80]:
import numpy as np
import cv2
import os

def load_images_from_folder(folder):
    """ Load binary images from the folder and normalize filenames to lowercase. """
    images = []
    filenames = []
    sorted_filenames = sorted(os.listdir(folder))  # Sort filenames

    for filename in sorted_filenames:
        normalized_filename = filename.lower()  # Normalize filename to lowercase
        img_path = os.path.join(folder, filename)  # Use original filename to form path
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img_binary = (img > 0).astype(np.uint8)  # Ensure binary format is standardized to 0 and 1
            images.append(img_binary)
            filenames.append(normalized_filename)  # Store normalized filename
    return images, filenames

In [81]:
def calculate_metrics(predictions, ground_truth):
    """ Calculate performance metrics comparing predictions to ground truth. """
    TP = np.sum((predictions == 1) & (ground_truth == 1))
    TN = np.sum((predictions == 0) & (ground_truth == 0))
    FP = np.sum((predictions == 1) & (ground_truth == 0))
    FN = np.sum((predictions == 0) & (ground_truth == 1))

    accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    intersection = np.sum((predictions == 1) & (ground_truth == 1))
    union = np.sum((predictions == 1) | (ground_truth == 1))
    iou = intersection / union if union > 0 else 0
    dice = 2 * intersection / (np.sum(predictions) + np.sum(ground_truth)) if (np.sum(predictions) + np.sum(ground_truth)) > 0 else 0

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'specificity': specificity,
        'f1_score': f1_score,
        'iou': iou,
        'dice': dice
    }

In [82]:
def evaluate_multiple_methods(method_folders, ground_truth_folder):
    """ Evaluate multiple segmentation methods against the same ground truth data. Normalize filenames to prevent mismatch. """
    ground_truth_images, gt_filenames = load_images_from_folder(ground_truth_folder)

    for method, folder in method_folders.items():
        print(f"Evaluating method: {method}")
        predictions, pred_filenames = load_images_from_folder(folder)

        if sorted(pred_filenames) != sorted(gt_filenames):
            print(f"Filename mismatch detected in {method}. Skipping this method.")
            continue  # Skip this method if filenames do not match

        metrics_list = []
        for pred, gt in zip(predictions, ground_truth_images):
            metrics = calculate_metrics(pred, gt)
            metrics_list.append(metrics)

        # Print average metrics
        avg_metrics = {k: np.mean([m[k] for m in metrics_list]) for k in metrics_list[0].keys()}
        print_metrics(method, avg_metrics)


In [83]:
def print_metrics(method, metrics):
    """ Print the calculated performance metrics for a given method. """
    print(f"Method: {method}")
    print(f"Accuracy: {metrics['accuracy']:.4f}, Precision: {metrics['precision']:.4f}, Recall: {metrics['recall']:.4f}, Specificity: {metrics['specificity']:.4f}, F1 Score: {metrics['f1_score']:.4f}")
    print(f"IoU: {metrics['iou']:.4f}, Dice Coefficient: {metrics['dice']:.4f}")


In [84]:
# Define method folders and the ground truth folder
method_folders = {
    "Adaptive Gaussian": "/content/drive/My Drive/AIA/mask/adaptive_gaussian_thresholding",
    "Adaptive Mean": "/content/drive/My Drive/AIA/mask/adaptive_mean_thresholding",
    "Adaptive Niblack": "/content/drive/My Drive/AIA/mask/adaptive_niblack_thresholding",
    "Otsu": "/content/drive/My Drive/AIA/mask/otsu_thresholding"
}
ground_truth_folder = '/content/drive/My Drive/AIA/teeth_mask'




In [85]:
# Evaluate the methods
evaluate_multiple_methods(method_folders, ground_truth_folder)

Evaluating method: Adaptive Gaussian
Method: Adaptive Gaussian
Accuracy: 0.8714, Precision: 0.5035, Recall: 0.5968, Specificity: 0.9119, F1 Score: 0.5318
IoU: 0.3724, Dice Coefficient: 0.5318
Evaluating method: Adaptive Mean
Method: Adaptive Mean
Accuracy: 0.8599, Precision: 0.4730, Recall: 0.6183, Specificity: 0.8957, F1 Score: 0.5215
IoU: 0.3628, Dice Coefficient: 0.5215
Evaluating method: Adaptive Niblack
Method: Adaptive Niblack
Accuracy: 0.8627, Precision: 0.4782, Recall: 0.5349, Specificity: 0.9118, F1 Score: 0.4904
IoU: 0.3329, Dice Coefficient: 0.4904
Evaluating method: Otsu
Method: Otsu
Accuracy: 0.8572, Precision: 0.4708, Recall: 0.7012, Specificity: 0.8792, F1 Score: 0.5484
IoU: 0.3910, Dice Coefficient: 0.5484
